In [302]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import ast
from textblob import TextBlob
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df_ted_main = pd.read_csv("data/ted_main.csv")
df_ted_transcripts = df = pd.read_csv("data/transcripts.csv")

In [303]:
df_ted_main.drop(['description', 'film_date', 'main_speaker', 'name', 'event', 'num_speaker', 'related_talks', 'speaker_occupation', 'title', 'url'], axis=1, inplace=True)
df_ted_main.head()

,comments,duration,languages,published_date,ratings,tags,views
0,4553,1164,60,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","['children', 'creativity', 'culture', 'dance',...",47227110
1,265,977,43,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","['alternative energy', 'cars', 'climate change...",3200520
2,124,1286,26,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","['computers', 'entertainment', 'interface desi...",1636292
3,200,1116,35,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","['MacArthur grant', 'activism', 'business', 'c...",1697550
4,593,1190,48,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","['Africa', 'Asia', 'Google', 'demo', 'economic...",12005869


In [304]:
# ----------------------------------------------------------
# 2. one hot encoding
# ----------------------------------------------------------

# one hot encoding tags

# iterate trough the data set
for i, row in df_ted_main.iterrows():
    # change string formed list into normal list
    element_list_form = ast.literal_eval(df_ted_main.at[i,'tags'])
    element_list_form = [i.strip() for i in element_list_form]
    # finish converting by replacing string version of a list for the list type
    df_ted_main.at[i,'tags'] = element_list_form

# one hot encoding with column named "tags"
# since "tags" is a Series on each row, we need to split them first and then we can do one hot encoding
df_ted_main = df_ted_main.drop('tags', 1).join(
    pd.get_dummies(
        pd.DataFrame(df_ted_main.tags.tolist()).stack()
    ).astype(int).sum(level=0)
)

df_ted_main.head()

,comments,duration,languages,published_date,ratings,views,3d printing,AI,AIDS,Addiction,...,wikipedia,wind energy,women,women in business,work,work-life balance,world cultures,writing,wunderkind,youth
0,4553,1164,60,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",47227110,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,265,977,43,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",3200520,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,124,1286,26,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",1636292,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200,1116,35,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",1697550,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,593,1190,48,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",12005869,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [305]:
# # one hot encoding "event" column

# def one_hot_encode(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     res = res.drop([feature_to_encode], axis=1)
#     return(res)

# df_ted_main = one_hot_encode(df_ted_main, 'event')
# df_ted_main.head()

In [306]:
#rating analysis

df_ted_main['ratings']=df_ted_main['ratings'].str.replace("'",'"')

df_ted_main=df_ted_main.merge(df_ted_main.ratings.apply(lambda x: pd.Series(pd.read_json(x)['count'].values,index=pd.read_json(x)['name'])), 
            left_index=True, right_index=True)

df_ted_main.drop('ratings', axis=1, inplace=True)

df_ted_main.head()

,comments,duration,languages,published_date,views,3d printing,AI,AIDS,Addiction,Africa,...,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
0,4553,1164,60,1151367060,47227110,0,0,0,0,0,...,387,242,7346,10581,300,10704,4439,1174,209,24924
1,265,977,43,1151367060,3200520,0,0,0,0,0,...,113,62,443,132,258,268,116,203,131,413
2,124,1286,26,1151367060,1636292,0,0,0,0,0,...,78,27,395,166,104,230,54,146,142,230
3,200,1116,35,1151367060,1697550,0,0,0,0,0,...,53,32,380,132,36,460,230,85,35,1070
4,593,1190,48,1151440680,12005869,0,0,0,0,1,...,110,72,5433,4606,67,2542,3736,248,61,2893


In [307]:
# sentiment analysis of transcript
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df_ted_transcripts['polarity'] = df_ted_transcripts['transcript'].apply(pol)
df_ted_transcripts['subjectivity'] = df_ted_transcripts['transcript'].apply(sub)
df_ted_transcripts.head()

,transcript,url,polarity,subjectivity
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,0.146452,0.462051
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,0.157775,0.424101
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,0.136579,0.475229
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,0.082928,0.439165
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,0.096483,0.450631


In [308]:
# merging both datasets and delete the rows that don't have a transcipt
df_ted_talks = pd.merge(df_ted_main, df_ted_transcripts, left_index=True, right_index=True)

In [309]:
# deleting the transcript and url columns because we don't need them.
del df_ted_talks['transcript']
del df_ted_talks['url']
df_ted_talks.head()

,comments,duration,languages,published_date,views,3d printing,AI,AIDS,Addiction,Africa,...,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity
0,4553,1164,60,1151367060,47227110,0,0,0,0,0,...,7346,10581,300,10704,4439,1174,209,24924,0.146452,0.462051
1,265,977,43,1151367060,3200520,0,0,0,0,0,...,443,132,258,268,116,203,131,413,0.157775,0.424101
2,124,1286,26,1151367060,1636292,0,0,0,0,0,...,395,166,104,230,54,146,142,230,0.136579,0.475229
3,200,1116,35,1151367060,1697550,0,0,0,0,0,...,380,132,36,460,230,85,35,1070,0.082928,0.439165
4,593,1190,48,1151440680,12005869,0,0,0,0,1,...,5433,4606,67,2542,3736,248,61,2893,0.096483,0.450631


In [310]:
# from sklearn.preprocessing import StandardScaler

# scaled_features = StandardScaler().fit_transform(result.values)

# result = pd.DataFrame(scaled_features, index=result.index, columns=result.columns)
# result

In [311]:
# # shuffle data set before splitting
# result = result.sample(frac=1).reset_index(drop=True)
# result

In [312]:
# from sklearn.model_selection import StratifiedShuffleSplit

# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in split.split(df_ted_talks, df_ted_talks["views"]):
#     df_train_set = df_ted_talks.loc[train_index]
#     df_test_set = df_ted_talks.loc[test_index]

# df_test_set["views"].value_counts() / len(df_test_set)

In [313]:
#We will shuffle the whole dataset first (df.sample(frac=1)) and then split our data set into the following parts:
# 60% - train set
# 20% - validation set
# 20% - test set

# df_train, df_val, df_test = np.split(df_ted_talks.sample(frac=1), [int(.6*len(df_ted_talks)), int(.8*len(df_ted_talks))])

In [314]:
# from sklearn.preprocessing import StandardScaler


# df_train_stand = StandardScaler().fit_transform(df_train.values)
# df_val_stand = StandardScaler().fit_transform(df_val.values)
# df_test_stand = StandardScaler().fit_transform(df_test.values)



# df_train = pd.DataFrame(df_train_stand, index=df_train.index, columns=df_train.columns)
# df_val = pd.DataFrame(df_val_stand, index=df_val.index, columns=df_val.columns)
# df_test = pd.DataFrame(df_test_stand, index=df_test.index, columns=df_test.columns)

In [315]:
# Splitting
train_set, test_set, train_labels, test_labels = train_test_split(df_ted_talks, df_ted_talks['views'], test_size = 1/7.0, random_state = 42)


In [316]:
# Standardization

from sklearn.preprocessing import StandardScaler

# Fit on training set only.
# https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i
scaler.fit(train_set)

# Apply transform to both the training set and the test set.
df_train_set_scaled = scaler.transform(train_set)
df_test_set_scaled = scaler.transform(test_set)

# convert to data frames
df_train_set = pd.DataFrame(df_train_set_scaled, index=train_set.index, columns=train_set.columns)
df_test_set = pd.DataFrame(df_test_set_scaled, index=test_set.index, columns=test_set.columns)



In [317]:
# PCA preprocessing - getting tag columns

# columns without PCA
columns_without_pca = ['views','subjectivity','polarity','Inspiring','Obnoxious','OK',
                                  'Jaw-dropping','Persuasive','Unconvincing','Fascinating','Informative','Confusing',
                                  'Longwinded','Courageous','Ingenious','Beautiful','Funny','comments','duration',
                                  'languages','published_date']

# get tag columns for both, training and test sets
tag_columns_train_set = df_train_set[df_train_set.columns.difference(columns_without_pca)]
tag_columns_test_set = df_test_set[df_test_set.columns.difference(columns_without_pca)]


##### DELETION #####
# all tag columns as a lists
tag_columns_train_set_list = list(tag_columns_train_set)
tag_columns_test_set_list = list(tag_columns_test_set)

# remove tag columns from original sets
df_train_set.drop(tag_columns_train_set_list, axis=1, inplace=True)
df_test_set.drop(tag_columns_test_set_list, axis=1, inplace=True)


# display all column names in the dataframe without tags
list(df_train_set)

['comments',
 'duration',
 'languages',
 'published_date',
 'views',
 'Funny',
 'Beautiful',
 'Ingenious',
 'Courageous',
 'Longwinded',
 'Confusing',
 'Informative',
 'Fascinating',
 'Unconvincing',
 'Persuasive',
 'Jaw-dropping',
 'OK',
 'Obnoxious',
 'Inspiring',
 'polarity',
 'subjectivity']

In [318]:
# PCA
pca = PCA(.80)

# fit PCA only on training set of tag columns
pca.fit(tag_columns_train_set)

# use PCA to transform both sets of tag columns
tag_columns_train_set = pca.transform(tag_columns_train_set)
tag_columns_test_set = pca.transform(tag_columns_test_set)


In [319]:
# combine: 
# PCA transformed training and test sets of tag columns, 
# with original training and test sets

# training set
feat_cols_train = ['feature'+str(i) for i in range(tag_columns_train_set.shape[1])]
# creating a dataframe from PCA results
df_pca_train = pd.DataFrame(tag_columns_train_set,columns=feat_cols_train)


# test set
feat_cols_test = ['feature'+str(i) for i in range(tag_columns_test_set.shape[1])]
# creating a dataframe from PCA results
df_pca_test = pd.DataFrame(tag_columns_test_set,columns=feat_cols_test)


# results of combining
df_train_set = df_train_set.join(df_pca_train)
df_test_set = df_test_set.join(df_pca_test)

In [324]:
df_pca_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2114 entries, 0 to 2113
Columns: 226 entries, feature0 to feature225
dtypes: float64(226)
memory usage: 3.6 MB


In [321]:

df_train_set.info()
df_test_set.info()

print('Training result Shape:', df_train_set.shape)
print('Testing result Shape:', df_test_set.shape)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1798 entries, 1241 to 860
Columns: 247 entries, comments to feature225
dtypes: float64(247)
memory usage: 3.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 1711 to 1987
Columns: 247 entries, comments to feature225
dtypes: float64(247)
memory usage: 693.9 KB
Training result Shape: (1798, 247)
Testing result Shape: (353, 247)
